In [1]:
!git clone https://github.com/chiayewken/Span-ASTE.git
# !cd Span-ASTE && git checkout 7cbf035
# !cd Span-ASTE && git checkout 92bc9a0
!cd Span-ASTE && git checkout 16c7937
!cp -a Span-ASTE/* .
!echo boto3==1.16.46 >> requirements.txt
!bash setup.sh

Cloning into 'Span-ASTE'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 168 (delta 44), reused 47 (delta 30), pack-reused 94
Receiving objects: 100% (168/168), 578.23 KiB | 23.13 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Note: checking out '16c7937'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 16c7937 Debug extra keys in SpanModel
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 31.7 MB/s 
     |███████████████

In [3]:
#@title Data Exploration
data_name = "14lap" #@param ["14lap", "14res", "15res", "16res"]

import sys
sys.path.append("aste")
from data_utils import Data

path = f"aste/data/triplet_data/{data_name}/train.txt"
data = Data.load_from_full_path(path)

for s in data.sentences[:3]:
    print("tokens:", s.tokens)
    for t in s.triples:
        print("target:", (t.t_start, t.t_end))
        print("opinion:", (t.o_start, t.o_end))
        print("label:", t.label)
    print()

tokens: ['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.']
target: (16, 17)
opinion: (15, 15)
label: LabelEnum.positive

tokens: ['it', 'is', 'of', 'high', 'quality', ',', 'has', 'a', 'killer', 'GUI', ',', 'is', 'extremely', 'stable', ',', 'is', 'highly', 'expandable', ',', 'is', 'bundled', 'with', 'lots', 'of', 'very', 'good', 'applications', ',', 'is', 'easy', 'to', 'use', ',', 'and', 'is', 'absolutely', 'gorgeous', '.']
target: (4, 4)
opinion: (3, 3)
label: LabelEnum.positive
target: (9, 9)
opinion: (8, 8)
label: LabelEnum.positive
target: (26, 26)
opinion: (25, 25)
label: LabelEnum.positive
target: (31, 31)
opinion: (29, 29)
label: LabelEnum.positive

tokens: ['Easy', 'to', 'start', 'up', 'and', 'does', 'not', 'overheat', 'as', 'much', 'as', 'other', 'laptops', '.']
target: (2, 3)
opinion: (0, 0)
label: LabelEnum.positive



In [8]:
# Download pretrained SpanModel weights
from pathlib import Path
template = "https://github.com/chiayewken/Span-ASTE/releases/download/v1.0.0/{}.tar"
url = template.format(data_name)
model_tar = Path(url).name
model_dir = Path(url).stem

!wget -nc $url
!tar -xf $model_tar

File ‘14lap.tar’ already there; not retrieving.



In [9]:
# Use pretrained SpanModel weights for prediction
import sys
sys.path.append("aste")
from pathlib import Path
from data_utils import Data, Sentence, SplitEnum
from wrapper import SpanModel

def predict_sentence(text: str, model: SpanModel) -> Sentence:
    path_in = "temp_in.txt"
    path_out = "temp_out.txt"
    sent = Sentence(tokens=text.split(), triples=[], pos=[], is_labeled=False, weight=1, id=0)
    data = Data(root=Path(), data_split=SplitEnum.test, sentences=[sent])
    data.save_to_path(path_in)
    model.predict(path_in, path_out)
    data = Data.load_from_full_path(path_out)
    return data.sentences[0]

text = "Did not enjoy the new Windows 8 and touchscreen functions ."
model = SpanModel(save_dir=model_dir, random_seed=0)
sent = predict_sentence(text, model)

for t in sent.triples:
    target = " ".join(sent.tokens[t.t_start:t.t_end+1])
    opinion = " ".join(sent.tokens[t.o_start:t.o_end+1])
    print()
    print(dict(target=target, opinion=opinion, sentiment=t.label))

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

################################################################################
################################################################################


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

{'span_model_unused_keys': dict_keys(['serialization_dir'])}
{'locals': ('span_extractor_type', 'endpoint')}
{'locals': ('use_span_width_embeds', True)}
{'ner_loss_fn': CrossEntropyLoss()}
{'unused_keys': dict_keys([])}
{'locals': {'self': ProperRelationExtractor(), 'make_feedforward': <function SpanModel.__init__.<locals>.make_feedforward at 0x7fcdb2c21ef0>, 'span_emb_dim': 1556, 'feature_size': 20, 'spans_per_word': 0.5, 'positive_label_weight': 1.0, 'regularizer': None, 'use_distance_embeds': True, 'use_pruning': True, 'kwargs': {}, 'vocab': Vocabulary with namespaces:  None__ner_labels, Size: 3 || None__relation_labels, Size: 3 || Non Padded Namespaces: {'*labels', '*tags'}, '__class__': <class 'span_model.models.relation_proper.ProperRelationExtractor'>}}
{'token_emb_dim': 768, 'span_emb_dim': 1556, 'relation_scorer_dim': 3240}
{'relation_loss_fn': CrossEntropyLoss()}


reading instances: 1it [00:00, 193.58it/s]


{'target': 'Windows 8', 'opinion': 'Did not enjoy', 'sentiment': <LabelEnum.negative: 'NEG'>}

{'target': 'touchscreen functions', 'opinion': 'Did not enjoy', 'sentiment': <LabelEnum.negative: 'NEG'>}

{'target': 'Windows 8', 'opinion': 'new', 'sentiment': <LabelEnum.neutral: 'NEU'>}


In [ ]:
# Train SpanModel from scratch
random_seed = 4
path_train = f"aste/data/triplet_data/{data_name}/train.txt"
path_dev = f"aste/data/triplet_data/{data_name}/dev.txt"
path_test = f"aste/data/triplet_data/{data_name}/test.txt"
save_dir = f"outputs/{data_name}/seed_{random_seed}"

model = SpanModel(save_dir=save_dir, random_seed=random_seed)
model.fit(path_train, path_dev)

{'weights_dir': PosixPath('outputs/14lap/seed_4/weights')}
{'random_seed': 4}
{'pytorch_seed': 4}
{'numpy_seed': 4}
{'train_data_path': PosixPath('/content/outputs/14lap/seed_4/temp_data/train.json')}
{'validation_data_path': PosixPath('/content/outputs/14lap/seed_4/temp_data/validation.json')}
{'test_data_path': PosixPath('/content/outputs/14lap/seed_4/temp_data/test.json')}
{'path_config': PosixPath('outputs/14lap/seed_4/config.jsonnet')}
{'command': 'cd /content && allennlp train outputs/14lap/seed_4/config.jsonnet --serialization-dir outputs/14lap/seed_4/weights --include-package span_model'}
2022-08-19 08:19:26,409 - INFO - allennlp.common.params - random_seed = 4
2022-08-19 08:19:26,409 - INFO - allennlp.common.params - numpy_seed = 4
2022-08-19 08:19:26,409 - INFO - allennlp.common.params - pytorch_seed = 4
2022-08-19 08:19:26,412 - INFO - allennlp.common.checks - Pytorch version: 1.7.0
2022-08-19 08:19:26,412 - INFO - allennlp.common.params - type = default
2022-08-19 08:19:26,

In [ ]:
# Evaluate SpanModel F1 Score
import json

path_pred = "pred.txt"
model.predict(path_in=path_test, path_out=path_pred)
results = model.score(path_pred, path_test)
print(json.dumps(results, indent=2))

{'command': 'cd /content && allennlp predict outputs/14lap/seed_4/weights/model.tar.gz /content/outputs/14lap/seed_4/temp_data/pred_in.json --predictor span_model --include-package span_model --use-dataset-reader  --output-file outputs/14lap/seed_4/temp_data/pred_out.json --cuda-device 0 --silent '}
################################################################################
################################################################################
{'locals': ('use_ner_embeds', False)}
{'locals': ('span_extractor_type', 'endpoint')}
{'locals': ('use_double_mix_embedder', False)}
{'locals': ('relation_head_type', 'proper')}
{'locals': ('use_span_width_embeds', True)}
{'ner_loss_fn': CrossEntropyLoss()}
{'unused_keys': dict_keys(['focal_loss_gamma', 'use_bi_affine_pruner', 'use_classify_mask_pruner', 'use_focal_loss', 'use_ner_scores_for_prune', 'use_ope_down_project', 'use_pair_feature_multiply', 'use_pairwise_down_project', 'use_span_loss_for_pruners', 'use_span_pair_aux_task